In [ ]:
%matplotlib qt
from bluesky import RunEngine
from bluesky.callbacks import LiveTable
from bluesky.plans import scan
from bluesky.plan_stubs import mv, open_run, close_run, create, \
    save, drop, rd, trigger, trigger_and_read, declare_stream
from ophyd.sim import det1, noisy_det, motor1
from databroker import Broker
from matplotlib.pyplot import ion
from bluesky.utils import install_nb_kicker
from bluesky.callbacks.best_effort import BestEffortCallback
from pprint import pprint
from IPython.display import display
install_nb_kicker()

ion()
RE = RunEngine()

db = Broker.named('temp')
RE.subscribe(db.insert)
bec = BestEffortCallback()
RE.subscribe(bec)

## Documents

Um dos principais objetivos do Bluesky é habilitar a coleta de dados e metadados. A meneira que ele faz isso é através dos **Documentos**nts.

Documentos são basicamente dicionários em Python com um esquema definido, ou seja, é um dicionário organizado de uma maneira específica e documentada. Esses **documentos** são gerados pela RunEngine durante a execução de planos e todos os dados e metadados estão organizados dentro dos **Documentos**.

### Runs e eventos

Planos podem gerar dados e metadados, porém como é possível organizar ou agrupar informações em Eventos e Runs?

In [ ]:
def one_run_one_event():
    yield from open_run()
    yield from trigger_and_read([det1, noisy_det])
    yield from close_run()

In [ ]:
RE(one_run_one_event())

Neste exemplo temos:
 - Uma tabela (uma Run)
 - Uma linha (um evento)
 - Duas colunas (além de seq_num e time), representando os dois detectores

In [ ]:
def one_run_n_events():
    yield from open_run()
    for i in range(4):
        yield from trigger_and_read([det1, noisy_det])
    yield from close_run()

In [ ]:
RE(one_run_n_events())

Neste exemplo temos:
 - Uma tabela (uma Run)
 - Quatro linhas (quatro eventos)
 - Duas colunas (além de seq_num e time), representando os dois detectores

In [ ]:
def multi_run():
    for i in range(2):
        yield from one_run_n_events()

In [ ]:
RE(multi_run())

Neste exemplo temos:
 - Duas tabelas (duas Runs)
 - Quatro linhas (quatro eventos por Run)
 - Duas colunas (além de seq_num e time), representando os dois detectores

   
Além disso, agora o retorno da função é uma tupla com multiplos uids, cada um relacionado com uma Run

A definição de uma Run é flexível, pois tudo depende de como você gostaria de organizar seus dados, seja para coleta ou análise.

### Documentos de uma Run

#### O Start Document
O Start document contém informações de metadados no início da run:
- time - Tempo de início do plano
- plan_name - Nome do plano
- uid - ID único de identificação da Run
- scan_id - id "mais amigável", porém não é necessáriamente único
- metadados

In [ ]:
uid, = RE(scan([det1], motor1, 0, 10, 6))

In [ ]:
last_run = db[uid]

In [ ]:
last_run.start

### Eventos
Vão guardar informações de um ou mais eventos:

In [ ]:
for event in last_run.events():
    display(event)
    break

In [ ]:
pprint(event.to_name_dict_pair())

### Metadata on Standard Plans

In [ ]:
(uid, ) = RE(scan([det1], motor1, -3, 3, 15, md={
    "motive": "Calibration",
    "sample": "Standard Sample"
}))
db[uid]

In [ ]:
last_run = db[-1]
last_run

### Start Event

- time: Horário de início do scan

- plan_name: Nome do plano executado, como 'scan' ou 'grid_scan'

- uid: ID único que identifica o scan realizado

- scan_id: Número inteiro amigável ao usuário, mas não necessáriamente único 

- Outros metadados inseridos pelo usuário ou plano

In [ ]:
last_run.start

### Stop Event

- time: Horário de fim do scan

- exit_status: Status do plano ao final ('abort', 'fail', 'success')

In [ ]:
last_run.stop

### Descriptors

In [ ]:
last_run.descriptors[0]

### Events

In [ ]:
last_run.table()

## Metadata Plan Stubs

In [ ]:
def plan_with_data():
    yield from open_run(md={})
    
    # Plan procedures
    yield from trigger_and_read([noisy_det])
    
    yield from close_run()

uid = RE(plan_with_data(), LiveTable(['noisy_det']))
db[-1].table()

In [ ]:
db[-1]

In [ ]:
def plan_with_data():
    yield from open_run(md={})
    
    # Plan procedures
    yield from trigger_and_read([noisy_det])
    yield from rd(motor1)
    
    yield from close_run()

uid = RE(plan_with_data(), LiveTable(['noisy_det', 'motor1']))
db[-1]

In [ ]:
def plan_with_data():
    yield from open_run(md={})
    yield from create(name="custom_data_stream")
    
    # Plan procedures
    yield from trigger(noisy_det)
    yield from rd(noisy_det)
    yield from rd(motor1)

    yield from save()
    yield from close_run()

uid = RE(plan_with_data(), LiveTable(['noisy_det']))
db[-1]

In [ ]:
db[-1].table("custom_data_stream")

create: Bundle future readings into a new Event document.

save: Close a bundle of readings and emit a completed Event document.

drop: Drop a bundle of readings without emitting a completed Event document.

subscribe: Subscribe the stream of emitted documents.

unsubscribe: Remove a subscription.


Plan Stubs

trigger: Trigger and acquisition.

read: Take a reading and add it to the current bundle of readings.

rd: Reads a single-value non-triggered object

Preprocessors -> What is a decorator

relative_set_decorator: Interpret 'set' messages on devices as relative to initial position.

reset_positions_decorator: Return movable devices to their initial positions at the end.

run_decorator: Enclose in 'open_run' and 'close_run' messages.

stage_decorator: 'Stage' devices (i.e., prepare them for use, 'arm' them) and then unstage.

finalize_decorator: try...finally helper

Suplemental Data



baseline_wrapper()

monitor_during_wrapper()

fly_during_wrapper()